In [24]:
import os 
import cv2 
import numpy as np
import tifffile 
from matplotlib import pyplot as plt 

In [15]:
fullstacks_dir  = '/home/zzf/dataset/IMC_Cell/Dice-XMBD/datas/melanoma/IMC_fullstacks/'
IMC_seg_cell_20_dir = '/mnt/mydisk/zzf/IMC_seg_cell_20/'
instace_dir = "/home/zzf/dataset/IMC_Cell/Dice-XMBD/datas/melanoma/IMC_segmentation/"

fullstack_ext = '_fullstacks.tiff'
seg_cell_20_ext = '_pred_Probabilities_cell_mask.tiff'

In [3]:
def vis(IMC_tif,seg_tif):
    C,H,W = IMC_tif.shape 
    DNA_c = IMC_tif[38]
    DNA_c[DNA_c>0]=255
    seg_tif[seg_tif>0]=255
    seg_tif = seg_tif[:H,:W]
    img1 = cv2.merge([DNA_c,DNA_c,DNA_c]).astype(np.uint8)
    img2 = cv2.merge([seg_tif,seg_tif,seg_tif]).astype(np.uint8)
    plt.imshow(np.concatenate((img1,img2),axis=1))

In [6]:
import random 
cnt = 0 
for file in os.listdir(IMC_seg_cell_20_dir):
    file = random.sample(list(os.listdir(IMC_seg_cell_20_dir)),1)[0]
    file = '_'.join(file.split('_')[:3])
    seg_tif = tifffile.imread(os.path.join(IMC_seg_cell_20_dir,file+seg_cell_20_ext))
    IMC_tif = tifffile.imread(os.path.join(fullstacks_dir,file+fullstack_ext))
    _,H,W = IMC_tif.shape
    tifffile.imwrite(os.path.join(IMC_seg_cell_20_dir,file+seg_cell_20_ext),seg_tif[:H,:W])
    # if cnt%10==0:
    print(file)
    print(seg_tif.shape,IMC_tif.shape)
    # vis(IMC_tif,seg_tif)
    cnt +=1 



20210329_33_ROI20
(512, 512) (41, 500, 500)
20210122_84F_ROI14
(1000, 1000) (41, 1000, 1000)
2021_67_ROI11
(1000, 1000) (41, 750, 750)
2021_67_ROI12
(1000, 1000) (41, 750, 750)
20210408_38_ROI10
(1000, 1000) (41, 1000, 1000)
20201124_87F_ROI9
(1000, 1000) (41, 1000, 1000)
20210305_85F_ROI14
(512, 512) (41, 500, 500)
2021_76_ROI16
(1000, 1000) (41, 1000, 1000)
20210408_45_ROI9
(512, 512) (41, 500, 500)
2021_79_ROI13
(512, 512) (41, 500, 500)
20210330_41_ROI11
(512, 512) (41, 500, 500)
20210305_36F_ROI12
(512, 512) (41, 500, 500)
2021_79_ROI11
(512, 512) (41, 500, 500)
2021_63_ROI12
(512, 512) (41, 500, 500)
2021_76_ROI18
(512, 512) (41, 500, 500)
2021_63_ROI13
(512, 512) (41, 500, 500)
20210329_33_ROI17
(512, 512) (41, 500, 500)
2021_67_ROI17
(1000, 1000) (41, 750, 750)
20210122_84F_ROI14
(1000, 1000) (41, 1000, 1000)
20210121_64F_ROI12
(1000, 1000) (41, 1000, 1000)
20210329_33_ROI17
(500, 500) (41, 500, 500)
2021_55_ROI12
(512, 512) (41, 500, 500)
2021_79_ROI16
(1000, 1000) (41, 1000, 

In [28]:
def center_pad_to_shape(img, size, cval=255):
    """Pad input image."""
    # rounding down, add 1
    pad_h = size[0] - img.shape[0]
    pad_w = size[1] - img.shape[1]
    pad_h = (pad_h // 2, pad_h - pad_h // 2)
    pad_w = (pad_w // 2, pad_w - pad_w // 2)
    if len(img.shape) == 2:
        pad_shape = (pad_h, pad_w)
    else:
        pad_shape = (pad_h, pad_w, (0, 0))
    img = np.pad(img, pad_shape, "constant", constant_values=cval)
    return img

def cropping_center(x, crop_shape, batch=False):
    """Crop an input image at the centre.

    Args:
        x: input array
        crop_shape: dimensions of cropped array
    
    Returns:
        x: cropped array
    
    """
    orig_shape = x.shape
    if not batch:
        h0 = int((orig_shape[0] - crop_shape[0]) * 0.5)
        w0 = int((orig_shape[1] - crop_shape[1]) * 0.5)
        x = x[h0 : h0 + crop_shape[0], w0 : w0 + crop_shape[1]]
    else:
        h0 = int((orig_shape[1] - crop_shape[0]) * 0.5)
        w0 = int((orig_shape[2] - crop_shape[1]) * 0.5)
        x = x[:, h0 : h0 + crop_shape[0], w0 : w0 + crop_shape[1]]
    return x

def get_bounding_box(img):
    """Get bounding box coordinate information."""
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    # due to python indexing, need to add 1 to max
    # else accessing will be 1px in the box, not out
    rmax += 1
    cmax += 1
    return [rmin, rmax, cmin, cmax]

def fix_mirror_padding(ann):
    """Deal with duplicated instances due to mirroring in interpolation
    during shape augmentation (scale, rotation etc.).
    
    """
    current_max_id = np.amax(ann)
    inst_list = list(np.unique(ann))
    inst_list.remove(0)  # 0 is background
    for inst_id in inst_list:
        inst_map = np.array(ann == inst_id, np.uint8)
        remapped_ids = measurements.label(inst_map)[0]
        remapped_ids[remapped_ids > 1] += current_max_id
        ann[remapped_ids > 1] = remapped_ids[remapped_ids > 1]
        current_max_id = np.amax(ann)
    return ann

## Generate HV map

In [50]:
import math
import numpy as np

import torch
import torch.nn.functional as F

from scipy import ndimage
from scipy.ndimage import measurements
from skimage import morphology as morph
import matplotlib.pyplot as plt

def gen_instance_hv_map(ann, crop_shape):
    """Input annotation must be of original shape.
    
    The map is calculated only for instances within the crop portion
    but based on the original shape in original image.

    Perform following operation:
    Obtain the horizontal and vertical distance maps for each
    nuclear instance.

    """
    orig_ann = ann.copy()  # instance ID map
    fixed_ann = fix_mirror_padding(orig_ann)
    # re-cropping with fixed instance id map
    crop_ann = cropping_center(fixed_ann, crop_shape)
    # TODO: deal with 1 label warning
    crop_ann = morph.remove_small_objects(crop_ann, min_size=30)

    x_map = np.zeros(orig_ann.shape[:2], dtype=np.float32)
    y_map = np.zeros(orig_ann.shape[:2], dtype=np.float32)

    inst_list = list(np.unique(crop_ann))
    inst_list.remove(0)  # 0 is background
    for inst_id in inst_list:
        inst_map = np.array(fixed_ann == inst_id, np.uint8)
        inst_box = get_bounding_box(inst_map)

        # expand the box by 2px
        # Because we first pad the ann at line 207, the bboxes
        # will remain valid after expansion
        inst_box[0] -= 2
        inst_box[2] -= 2
        inst_box[1] += 2
        inst_box[3] += 2

        inst_map = inst_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]

        if inst_map.shape[0] < 2 or inst_map.shape[1] < 2:
            continue

        # instance center of mass, rounded to nearest pixel
        inst_com = list(measurements.center_of_mass(inst_map))

        inst_com[0] = int(inst_com[0] + 0.5)
        inst_com[1] = int(inst_com[1] + 0.5)

        inst_x_range = np.arange(1, inst_map.shape[1] + 1)
        inst_y_range = np.arange(1, inst_map.shape[0] + 1)
        # shifting center of pixels grid to instance center of mass
        inst_x_range -= inst_com[1]
        inst_y_range -= inst_com[0]

        inst_x, inst_y = np.meshgrid(inst_x_range, inst_y_range)

        # remove coord outside of instance
        inst_x[inst_map == 0] = 0
        inst_y[inst_map == 0] = 0
        inst_x = inst_x.astype("float32")
        inst_y = inst_y.astype("float32")

        # normalize min into -1 scale
        if np.min(inst_x) < 0:
            inst_x[inst_x < 0] /= -np.amin(inst_x[inst_x < 0])
        if np.min(inst_y) < 0:
            inst_y[inst_y < 0] /= -np.amin(inst_y[inst_y < 0])
        # normalize max into +1 scale
        if np.max(inst_x) > 0:
            inst_x[inst_x > 0] /= np.amax(inst_x[inst_x > 0])
        if np.max(inst_y) > 0:
            inst_y[inst_y > 0] /= np.amax(inst_y[inst_y > 0])

        ####
        x_map_box = x_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]
        x_map_box[inst_map > 0] = inst_x[inst_map > 0]

        y_map_box = y_map[inst_box[0] : inst_box[1], inst_box[2] : inst_box[3]]
        y_map_box[inst_map > 0] = inst_y[inst_map > 0]

    hv_map = np.dstack([x_map, y_map])
    return hv_map
for file in os.listdir(IMC_seg_cell_20_dir):
    print(file,file in os.listdir('/mnt/mydisk/zzf/IMC_seg_cell_hv/'))
    # if file in os.listdir('/mnt/mydisk/zzf/IMC_seg_cell_hv/'):
        # continue
    
    # file = random.sample(list(os.listdir(IMC_seg_cell_20_dir)),1)[0]
    file = '_'.join(file.split('_')[:3])
    inst_ann = tifffile.imread(os.path.join(instace_dir,file+seg_cell_20_ext))
    crop_shape = inst_ann.shape
    print(crop_shape)
    hv = gen_instance_hv_map(inst_ann,crop_shape)
    print(f"/mnt/mydisk/zzf/IMC_seg_cell_hv/{file+seg_cell_20_ext}",hv.shape,hv.max(),hv.min())
    tifffile.imwrite(f"/mnt/mydisk/zzf/IMC_seg_cell_hv/{file+seg_cell_20_ext}",hv)


20210329_33_ROI13_pred_Probabilities_cell_mask.tiff False
(512, 512)


/tmp/ipykernel_1786687/713382471.py:59: DeprecationWarning: Please import `label` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  remapped_ids = measurements.label(inst_map)[0]
/tmp/ipykernel_1786687/3507934882.py:53: DeprecationWarning: Please import `center_of_mass` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  inst_com = list(measurements.center_of_mass(inst_map))


/mnt/mydisk/zzf/IMC_seg_cell_hv/20210329_33_ROI13_pred_Probabilities_cell_mask.tiff (512, 512, 2) 1.0 -1.0
2021_65_ROI16_pred_Probabilities_cell_mask.tiff False
(512, 512)
/mnt/mydisk/zzf/IMC_seg_cell_hv/2021_65_ROI16_pred_Probabilities_cell_mask.tiff (512, 512, 2) 1.0 -1.0
20210120_73F_ROI11_pred_Probabilities_cell_mask.tiff False
(1024, 1024)
/mnt/mydisk/zzf/IMC_seg_cell_hv/20210120_73F_ROI11_pred_Probabilities_cell_mask.tiff (1024, 1024, 2) 1.0 -1.0
20210115_12F_ROI10_pred_Probabilities_cell_mask.tiff False
(1024, 1024)
/mnt/mydisk/zzf/IMC_seg_cell_hv/20210115_12F_ROI10_pred_Probabilities_cell_mask.tiff (1024, 1024, 2) 1.0 -1.0
20210121_64F_ROI11_pred_Probabilities_cell_mask.tiff False
(1024, 1024)
/mnt/mydisk/zzf/IMC_seg_cell_hv/20210121_64F_ROI11_pred_Probabilities_cell_mask.tiff (1024, 1024, 2) 1.0 -1.0
20210408_38_ROI11_pred_Probabilities_cell_mask.tiff False
(1024, 1024)
/mnt/mydisk/zzf/IMC_seg_cell_hv/20210408_38_ROI11_pred_Probabilities_cell_mask.tiff (1024, 1024, 2) 1.0 -1.0